<a href="https://colab.research.google.com/github/nishantsinha5/R-Tutorials/blob/master/IMDLIB_Tmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Connect Google drive with Google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
pip install imdlib

In [13]:
# Download the IMD raw data into the google drive
import imdlib as imd
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon

path = "/content/drive/MyDrive/Colab Notebooks"

start_yr = 1951
end_yr = 2023
variable = 'tmax' # other options are ('tmin'/ 'tmax')

imd.get_data(variable, start_yr, end_yr, fn_format='yearwise', file_dir=path)
data = imd.open_data(variable, start_yr, end_yr,'yearwise', path)
ds = data.get_xarray()
print(ds)



Downloading: maxtemp for year 1951
Downloading: maxtemp for year 1952
Downloading: maxtemp for year 1953
Downloading: maxtemp for year 1954
Downloading: maxtemp for year 1955
Downloading: maxtemp for year 1956
Downloading: maxtemp for year 1957
Downloading: maxtemp for year 1958
Downloading: maxtemp for year 1959
Downloading: maxtemp for year 1960
Downloading: maxtemp for year 1961
Downloading: maxtemp for year 1962
Downloading: maxtemp for year 1963
Downloading: maxtemp for year 1964
Downloading: maxtemp for year 1965
Downloading: maxtemp for year 1966
Downloading: maxtemp for year 1967
Downloading: maxtemp for year 1968
Downloading: maxtemp for year 1969
Downloading: maxtemp for year 1970
Downloading: maxtemp for year 1971
Downloading: maxtemp for year 1972
Downloading: maxtemp for year 1973
Downloading: maxtemp for year 1974
Downloading: maxtemp for year 1975
Downloading: maxtemp for year 1976
Downloading: maxtemp for year 1977
Downloading: maxtemp for year 1978
Downloading: maxtemp

In [9]:
# Provide the alttitude & Longitude of a point for which the data is required
#  And save the data into CSV file

lat = 20.03 #lattitude of point
lon = 77.23 #longitude of point
data.to_csv('data.csv', lat, lon, path)

# New Section

In [14]:
# Save CSV files for multiple points

# Provide lat and long in a list
latLong = [[20.70,77.04],[13.08,77.58],[11.01,76.94],[23.21,79.95],[21.51,70.45],[30.90,75.81],[28.64,77.16],[32.11,76.54],[19.26,76.77],[23.43,85.32],[24.59,73.73],[22.76,88.43],[29.02,79.49],[18.79,78.91],[20.30,85.82],[10.81,76.20],[21.24,81.71],[23.26,77.41]]

for points in latLong:
  lat = points[0]
  lon = points[1]

  data.to_csv('Tmax.csv', lat, lon, path)
  print ("data save for ",points)

data save for  [20.7, 77.04]
data save for  [13.08, 77.58]
data save for  [11.01, 76.94]
data save for  [23.21, 79.95]
data save for  [21.51, 70.45]
data save for  [30.9, 75.81]
data save for  [28.64, 77.16]
data save for  [32.11, 76.54]
data save for  [19.26, 76.77]
data save for  [23.43, 85.32]
data save for  [24.59, 73.73]
data save for  [22.76, 88.43]
data save for  [29.02, 79.49]
data save for  [18.79, 78.91]
data save for  [20.3, 85.82]
data save for  [10.81, 76.2]
data save for  [21.24, 81.71]
data save for  [23.26, 77.41]


In [ ]:
#  Provide the Geojson file of a catchment or polygon to dowlnaod all the gridded data lying into that polygon

geojson_file = '/content/drive/MyDrive/IMD/Test_geojson.geojson'


url="https://drive.google.com/file/d/111XvmUzvTlhY2lbFMseGVhZQh4pisFXQ/view?usp=sharing"
url2='https://drive.google.com/uc?id=' + url.split('/')[-2]

points_df = pd.read_csv(url2)


geometry = [Point(xy) for xy in zip(points_df['Long'], points_df['Lat'])]

# Creating the GeoDataFrame
gdf_points = gpd.GeoDataFrame(points_df, geometry=geometry)

# Set a CRS (coordinate reference system), EPSG:4326 is WGS84 Lat/Long
gdf_points.set_crs(epsg=4326, inplace=True)


gdf_polygon = gpd.read_file(geojson_file)

# Ensure both GeoDataFrames use the same CRS
if gdf_points.crs != gdf_polygon.crs:
    gdf_points = gdf_points.to_crs(gdf_polygon.crs)

gdf_list = []
for row in range (len(gdf_polygon)):
    points_in_polygon = gdf_points[gdf_points.within(gdf_polygon.iloc[row].geometry)]
    gdf_list.append(points_in_polygon)

final_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

final_df = final_gdf[["Name","Lat","Long"]]
final_df.to_csv("Master_file.csv")

for index, row in final_df.iterrows():
    lat = row['Lat']
    lon = row['Long']
    data.to_csv('test.csv', lat, lon, path)
    print ("data save for " + str(lat)+ "_" +str(lon))